### Embedding 계층 구현

In [4]:
import numpy as np
W = np.arange(21).reshape(7, 3)
x = np.array([1,0,0,0,0,0,0]) #임의 onehot encoding
print(W)
h = np.dot(x, W)
print(h) # 행렬곱이나
print(W[0]) # 선택이나 같은 값 [0 1 2] return

[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]
 [15 16 17]
 [18 19 20]]
[0 1 2]
[0 1 2]


In [5]:
W[2]

array([6, 7, 8])

In [6]:
W[5]

array([15, 16, 17])

In [7]:
idx = np.array([1, 0, 3, 0]) # you say goodbye and i say hello . 에서 say, you, and, you 를 뽑은 것 과 같음. 여기서 배치처리 하면서 들어온 중복된 you는 역전파 미분할 때 문제가 됨ㅊ 
W[[1,0,3,0]] # 인텍스 1 0 3 0 만 꺼내서 새로운 배열을 만드는 python 기법

array([[ 3,  4,  5],
       [ 0,  1,  2],
       [ 9, 10, 11],
       [ 0,  1,  2]])

In [26]:
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None

    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out

    # 옳은 backward (simplified)
    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
        np.add.at(dW, self.idx, dout)
        return None    

#     옳은 backward
#     def backward(self, dout):
#         dW, = self.grads
#         dW[...] = 0
#         for i, word_id in enumerate(self.idx):
#             dW[word_id] += dout[i]
#         return None
    
#     옳지 않은 backward    
#     def backward(self, dout):
#         dW, = self.grads
#         dW[...] = 0
#         dW[self.idx] = dout
#         return None    

In [27]:
embed = Embedding(W)
out = embed.forward([1,0,3,0]) # 선택하기
print(embed.idx) #선택된 idx 기록되어있음
print(out)

[1, 0, 3, 0]
[[ 3  4  5]
 [ 0  1  2]
 [ 9 10 11]
 [ 0  1  2]]


In [28]:
dout = np.array([[1,1,1], # 1번째에 반영
                 [2,2,2], # 0번째에 반영
                 [3,3,3], # 3번쨰에 반영
                 [4,4,4]])# 다시 0번쨰에 반영
embed.backward(dout)
embed.grads

'''
[array([[6, 6, 6], # [2 2 2] + [4 4 4] 가 올바르게 반영된 모습
        [1, 1, 1],
        [0, 0, 0],
        [3, 3, 3],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])]
'''

[array([[6, 6, 6],
        [1, 1, 1],
        [0, 0, 0],
        [3, 3, 3],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])]

In [7]:
    # 옳지 않은 backward 
    # 같은 행이 선택되면 미분값이 이 전 미분값을 덮어씌우버림
    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
        dW[self.idx] = dout
        return None

In [8]:
    # 옳은 backward
    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
        for i, word_id in enumerate(self.idx):
            dW[word_id] += dout[i]
        return None

In [9]:
    # 옳은 backward (simplified)
    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
        np.add.at(dW, self.idx, dout)
        return None

In [25]:
dW = np.zeros(5)
idx = np.array([0,1,2,0])
np.add.at(dW, idx, 1) # add.at(list, index, 1) 은 list 의 given index에 1을 더하라는 뜻. 0이 두번 있으므로 [2. 1. 1. 0. 0.]가 return
print(dW)

[2. 1. 1. 0. 0.]
